In [3]:
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp

In [1]:
def truncated_ladder_ops(cutoff):
    """Return a, a† in the truncated Fock basis (cutoff levels)."""
    a = np.zeros((cutoff, cutoff), dtype=complex)
    for n in range(cutoff-1):
        a[n, n+1] = np.sqrt(n+1)
    adag = a.conj().T
    return a, adag

def q_p_ops(cutoff):
    """Canonical q, p with ħ=1 in the truncated basis."""
    a, adag = truncated_ladder_ops(cutoff)
    q = (adag + a) / np.sqrt(2.0)
    p = 1j * (adag - a) / np.sqrt(2.0)
    return q, p

def aho_W_derivs(q, m, g):
    """Given matrix q, build W'(q)=m q + g q^3 and W''(q)=m + 3 g q^2."""
    q2 = q @ q
    q3 = q2 @ q
    Wp  = m * q + g * q3
    Wpp = m * np.eye(q.shape[0]) + 3.0 * g * q2
    return Wp, Wpp

def kron3(A, B, C):
    return np.kron(np.kron(A, B), C)

def aho_susy_paulis(cutoff=4, m=1.0, g=0.1, wire_order=(0,1,2)):
    """
    Build H = 1/2 [ p^2 + (W')^2 + W'' ⊗ σz ] for AHO
    with boson cutoff 'cutoff' (uses ⌈log2(cutoff)⌉ qubits) and 1 fermion qubit.
    Returns:
      - pl_op: a PennyLane Operator that is the Pauli-string sum
      - terms: list of (coeff, word) for easy inspection
    """
    # --- boson (cutoff-dimensional) operators ---
    q, p = q_p_ops(cutoff)
    Wp, Wpp = aho_W_derivs(q, m, g)

    Hb = 0.5 * (p @ p + Wp @ Wp)            # purely bosonic part
    Zf = np.array([[1,0],[0,-1]], dtype=complex)  # σz on fermion

    # --- total Hilbert space = boson ⊗ fermion ---
    # Encode boson in nb = ceil(log2(cutoff)) qubits via binary encoding.
    nb = int(np.ceil(np.log2(cutoff)))
    if 2**nb != cutoff:
        # pad bosonic operators to dimension 2**nb (embedding)
        pad = 2**nb - cutoff
        def pad_to_pow2(M):
            Z = np.zeros((2**nb, 2**nb), dtype=complex)
            Z[:cutoff, :cutoff] = M
            return Z
        Hb_emb  = pad_to_pow2(Hb)
        Wpp_emb = pad_to_pow2(Wpp)
    else:
        Hb_emb, Wpp_emb = Hb, Wpp

    Id_b = np.eye(2**nb, dtype=complex)
    Id_f = np.eye(2, dtype=complex)

    H_tot = np.kron(Hb_emb, Id_f) + 0.5 * np.kron(Wpp_emb, Zf)

    # --- Decompose into Pauli words on nb + 1 qubits ---
    # wire_order: place boson qubits first, fermion last by default.
    wires = list(wire_order)
    assert len(wires) == nb + 1, "wire_order must have nb (boson) + 1 (fermion) entries"

    pl_op = qml.pauli_decompose(H_tot, wire_order=wires)

    # Extract nice (coeff, string) list
    coeffs, ops = [], []
    try:
        # For modern PennyLane, pauli_decompose returns a Sum/LinearCombination
        coeffs, factors = pl_op.terms()
        words = []
        for fac in factors:
            # fac is a Tensor of Pauli ops; turn into a simple string like "X0 Y1 Z2"
            pieces = []
            for pa in fac.operands if hasattr(fac, "operands") else [fac]:
                name = pa.name  # 'PauliX', 'PauliY', 'PauliZ', 'Identity'
                w = pa.wires.tolist()
                tag = {"PauliX":"X", "PauliY":"Y", "PauliZ":"Z", "Identity":"I"}[name]
                pieces.append(f"{tag}{w[0]}")
            words.append(" ".join(sorted(pieces, key=lambda s: int(s[1:]))))
        terms = list(zip([float(c) for c in coeffs], words))
    except Exception:
        # Fallback: just return the operator
        terms = None

    return pl_op, terms


In [7]:
# --- Example usage ---
pl_op, terms = aho_susy_paulis(cutoff=8, m=1.0, g=1.0, wire_order=(0,1,2,3))

print("Number of Pauli terms:", len(terms))
for c, w in terms:
    print(f"{c:+.8f} * {w}")

Number of Pauli terms: 34
+113.31250000 * I0 I1 I2 I3
+5.75000000 * I0 I1 I2 Z3
+88.18906352 * I0 X1 I2 I3
+2.96656305 * I0 X1 I2 Z3
-0.07150582 * I0 X1 Z2 I3
-0.38227337 * I0 X1 Z2 Z3
-9.06250000 * I0 Z1 I2 I3
-0.75000000 * I0 Z1 I2 Z3
-25.06250000 * I0 Z1 Z2 I3
-0.75000000 * I0 Z1 Z2 Z3
+28.05664664 * X0 I1 I2 I3
-6.76713867 * X0 I1 Z2 I3
+42.15674833 * X0 X1 I2 I3
+1.48804454 * X0 X1 I2 Z3
-14.14667579 * X0 X1 Z2 I3
-0.18900644 * X0 X1 Z2 Z3
-16.58971261 * X0 Z1 I2 I3
+1.11774278 * X0 Z1 Z2 I3
+39.09969292 * Y0 Y1 I2 I3
+1.48804454 * Y0 Y1 I2 Z3
-12.76667137 * Y0 Y1 Z2 I3
-0.18900644 * Y0 Y1 Z2 Z3
-80.56250000 * Z0 I1 I2 I3
-2.25000000 * Z0 I1 I2 Z3
-13.81250000 * Z0 I1 Z2 I3
-0.75000000 * Z0 I1 Z2 Z3
-74.04041513 * Z0 X1 I2 I3
-1.51767431 * Z0 X1 I2 Z3
-7.97834659 * Z0 X1 Z2 I3
-0.00595520 * Z0 X1 Z2 Z3
-16.68750000 * Z0 Z1 I2 I3
-0.75000000 * Z0 Z1 I2 Z3
+34.06250000 * Z0 Z1 Z2 I3
+0.75000000 * Z0 Z1 Z2 Z3


In [6]:
import scipy.linalg as la

H_mat = qml.matrix(pl_op)   # or sparse=True for CSR matrix

evals, evecs = la.eigh(H_mat)
print("Ground energy =", evals[0])


Ground energy = -0.16478526068502194


In [8]:
# Jupyter-ready
import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from scipy.sparse.linalg import eigsh

# ---------- Utilities: Pauli building blocks ----------
def P0(w):  # (I+Z)/2 projector onto |0>
    return 0.5 * (qml.I(w) + qml.PauliZ(w))

def P1(w):  # (I-Z)/2 projector onto |1>
    return 0.5 * (qml.I(w) - qml.PauliZ(w))

def S_plus(w):
    # (X - iY)/2
    return 0.5 * (qml.PauliX(w) - 1j * qml.PauliY(w))

def S_minus(w):
    # (X + iY)/2
    return 0.5 * (qml.PauliX(w) + 1j * qml.PauliY(w))

def bitstring(n, q):
    return [(n >> k) & 1 for k in range(q)]  # little-endian bits [b0,b1,...]

# ---------- Map |n><m| to a Pauli operator product sentence ----------
def ketbra_pauli_sentence(n, m, wires):
    """Return a PennyLane operator (sum of Pauli products) implementing |n><m|
    on the given 'wires' (binary-encoded)."""
    q = len(wires)
    bn = bitstring(n, q)
    bm = bitstring(m, q)

    # Build the tensor product factor-by-factor.
    # Each factor can be a sum (e.g., P0, P1), so we expand distributively.
    terms = [(1.0, qml.I(wires[0]).__class__(wires=[]))]  # start with scalar*Identity (dummy)

    # We'll accumulate as (coeff, op) pairs where op is a qml operation product
    for k, w in enumerate(wires):
        ak, bk = bn[k], bm[k]
        if ak == bk:
            # projector P0 or P1
            factor = P0(w) if ak == 0 else P1(w)
            # factor is sum of Pauli products; expand
            # P0 = 0.5 I + 0.5 Z ; P1 = 0.5 I - 0.5 Z
            factor_terms = [(0.5, qml.I(w)), (0.5 if ak==0 else -0.5, qml.PauliZ(w))]
        else:
            # bit flip: 0->1 means sigma+, 1->0 means sigma-
            factor_op = S_plus(w) if (bk == 1 and ak == 0) else S_minus(w)
            # S± = 0.5 X ∓ i*0.5 Y
            if factor_op is S_plus(w):   # (X - iY)/2
                factor_terms = [(0.5, qml.PauliX(w)), (-0.5j, qml.PauliY(w))]
            else:                         # (X + iY)/2
                factor_terms = [(0.5, qml.PauliX(w)), (0.5j, qml.PauliY(w))]

        # Distribute factor_terms across current terms
        new_terms = []
        for c1, op1 in terms:
            for c2, op2 in factor_terms:
                new_terms.append((c1 * c2, qml.prod(op1, op2)))
        terms = new_terms

    # Combine identical Pauli words (same op structure) to compress
    # Use qml.pauli.PauliSentence for canonical collection/simplification
    sent = qml.pauli.PauliSentence({})
    for c, op in terms:
        # qml.prod gives a CompositeOp; convert to PauliSentence if possible
        ps = qml.pauli.pauli_sentence(op)
        sent = sent + (c * ps)

    return sent

# ---------- Build a and adag as Pauli sentences (binary encoding) ----------
def boson_ladder_as_pauli(d, wires):
    """Return (a_sentence, adag_sentence) for a single bosonic mode with cutoff d."""
    a = qml.pauli.PauliSentence({})
    for n in range(d - 1):
        sent = ketbra_pauli_sentence(n, n + 1, wires)
        a = a + (np.sqrt(n + 1) * sent)
    adag = a.adjoint()  # exact Hermitian adjoint at sentence level
    return a, adag

# ---------- From sentences to PennyLane Hamiltonian ----------
def sentence_to_hamiltonian(sent):
    coeffs = []
    ops = []
    for word, c in sent.items():
        coeffs.append(c)
        # word is a PauliWord -> convert to op product
        if len(word) == 0:
            ops.append(qml.Identity(0))  # global identity (wire will be ignored by PennyLane)
        else:
            # Build product in a stable wire order
            pieces = []
            for w, p in word.items():
                if p == "X":
                    pieces.append(qml.PauliX(w))
                elif p == "Y":
                    pieces.append(qml.PauliY(w))
                elif p == "Z":
                    pieces.append(qml.PauliZ(w))
            ops.append(qml.prod(*pieces))
    return qml.Hamiltonian(pnp.array(coeffs), ops)

# ---------- AHO Hamiltonian: H = 1/2 p^2 + 1/2 x^2 + lambda x^4 ----------
def aho_hamiltonian_paulis(cutoff_d=4, lam=0.1, wires=None):
    """
    Returns a PennyLane Hamiltonian (sum of Pauli products) for a single-mode AHO
    with cutoff d, using binary encoding on 'wires'.
    """
    if wires is None:
        q = int(np.ceil(np.log2(cutoff_d)))
        wires = list(range(q))

    a, adag = boson_ladder_as_pauli(cutoff_d, wires)
    # x = (a + adag)/sqrt(2),  p = i(adag - a)/sqrt(2)
    x = (a + adag) * (1 / np.sqrt(2))
    p = (adag - a) * (1j / np.sqrt(2))

    # Build H term-by-term using sentence algebra; then convert to qml.Hamiltonian
    H_sent = 0.5 * (p @ p) + 0.5 * (x @ x) + lam * (x @ x @ x @ x)
    return sentence_to_hamiltonian(H_sent.simplify()), wires

# ---------- Optional: exact ground-state via sparse eigensolver ----------
def exact_ground_energy(H_pl, wires):
    """
    Convert the PennyLane Hamiltonian to a SciPy sparse matrix and compute the
    lowest eigenvalue with eigsh (matrix is still sparse, not dense).
    """
    # Note: H_pl.sparse_matrix builds a sparse CSR matrix of size 2^n x 2^n
    sp = H_pl.sparse_matrix(wires=wires)
    # Use eigsh on the Hermitian sparse matrix
    evals, _ = eigsh(sp, k=1, which="SA")  # smallest algebraic
    return float(np.real(evals[0]))


# ---------------------- Example usage ----------------------
# Build AHO with cutoff d=4 (i.e., 2 qubits), lambda=0.1
H_aho, boson_wires = aho_hamiltonian_paulis(cutoff_d=4, lam=0.1)

print("Number of qubits:", len(boson_wires))
print("Number of Pauli terms:", len(H_aho.ops))
E0 = exact_ground_energy(H_aho, boson_wires)
print("Exact ground-state energy (cutoff d=4):", E0)


AttributeError: 'PauliSentence' object has no attribute 'adjoint'